# MongoDB and Python

MongoDB is a 'NoSQL database' with support for high-performance document-oriented storage and queries, sharding, and replication.

Terminology:

- A **document** is a single JSON-like object stored in MongoDB
- A **collection** is a respository of documents which may have one or more indexes on them
- A **database** is a group of collections and indexes 


To get started, we'll install the `pymongo` driver and the `dnspython` modules to allow us to use the "mongodb+srv://" URLs to connect to MongoDB:

In [1]:
!pip install -U pymongo dnspython

Error processing line 1 of /Users/sridharmundra/Library/Python/3.8/lib/python/site-packages/pyston_autoload.pth:

  Traceback (most recent call last):
    File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<string>", line 1, in <module>
  ImportError: dlopen(/Users/sridharmundra/Library/Python/3.8/lib/python/site-packages/pyston.cpython-38-darwin.so, 0x0002): symbol not found in flat namespace (__PyAsyncGenValueWrapperNew)

Remainder of file ignored
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


## Connecting and accessing databases and collections

In [5]:
import pymongo
username = 'class'
password = 'classword'
host = 'cluster0.bpduwlt.mongodb.net'
dbname = 'training'
cli = pymongo.MongoClient(
    f'mongodb+srv://{username}:{password}@{host}'#'/{dbname}'
)
#cli = pymongo.MongoClient('mongodb://localhost:27017/class')

ModuleNotFoundError: No module named 'pymongo'

In [3]:
cli

NameError: name 'cli' is not defined

In [ ]:
db = cli['training']
db

In [ ]:
cli.training

In [ ]:
for cname in db.list_collection_names():
    db[cname].drop()

## Inserting data

https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html

In [ ]:
res = db.roster.insert_one({
    'name': 'Rick Copeland',
    'email': 'rick@arborian.com',
    'role': 'Instructor',
})
res

In [ ]:
res.inserted_id

## Querying data

[Additional documentation: query operators](https://docs.mongodb.com/manual/reference/operator/query/)

[Additional documentation: find](http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find)

[Additional documentation: find_one](http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find_one)

In [ ]:
cursor = db.roster.find({'name': 'Rick Copeland'})
cursor

In [ ]:
list(cursor)

In [ ]:
for doc in db.roster.find({'name': {'$eq': 'Rick Copeland'}}):
    print(doc)

In [ ]:
for item in db.roster.find({'name': {'$gte': 'Rick Copeland'}}):
    print(item)

In [ ]:
doc = db.roster.find_one({'_id': item['_id']})
doc

In [ ]:
import re
db.roster.find_one({'role': re.compile('^Ins')})  # SQL equivalent: LIKE 'Ins%'

## Updating data

[Additonal documentation: update operators](https://docs.mongodb.com/manual/reference/operator/update/)

[Additional documentation: replace](http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.replace_one)

[Additional documentation: update](http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.update_one)

In [ ]:
doc['email'] = 'rick446@arborian.com'
db.roster.replace_one(
    {'_id': doc['_id']},
    doc
)

In [ ]:
doc = db.roster.find_one()
doc

In [ ]:
db.roster.update_one(
    {'_id': doc['_id']},
    {'$set': {'email': 'rick@arborian.com'}}
)

In [ ]:
doc = db.roster.find_one()
doc

## Atomic find/modify

In [ ]:
coll = db.roster
doc = coll.find_one_and_update(
    {'name': 'Rick Copeland'},
    {'$inc': {'classes': 1}},
    return_document=pymongo.ReturnDocument.AFTER,
    upsert=True,
)
doc

In [ ]:
coll = db.roster
doc = coll.find_one_and_update(
    {'name': 'Richard Copeland'},
    {'$inc': {'classes': 1}},
    return_document=pymongo.ReturnDocument.AFTER,
    upsert=True,
)
doc

In [ ]:
list(db.roster.find())

Psuedocode for locking using MongoDB

```python

{
    waiters: [
        ...
    ]
}

lock = db.lock.find_one_and_update(
    {_id: 'mylock'},
    {'$push': {'waiters': my_unique_id}}
    return_document=pymongo.ReturnDocument.AFTER
)

if lock.waiters[0] == my_id:
    I got it!
```

## Delete

In [ ]:
import re

res = coll.delete_one({'name': re.compile(r'^Ri')})
res

In [ ]:
list(coll.find())

In [ ]:
second_res = coll.delete_many({'name': re.compile(r'^Ri')})

In [ ]:
res.deleted_count

In [ ]:
second_res.deleted_count

In [ ]:
list(coll.find())

In [ ]:
import pandas as pd
stock = pd.read_csv('data/closing-prices.csv', parse_dates=[0])
stock.head()

In [ ]:
records = stock.to_dict(orient='records')

In [ ]:
records[0]

In [ ]:
db.stock.insert_many(records)

In [ ]:
len(_36.inserted_ids)

In [ ]:
db.stock.find_one()

In [ ]:
from datetime import datetime
db.stock.find_one({'Unnamed: 0': datetime(2014,1,2)})

Open [PyMongo Lab](./pymongo-lab.ipynb)